In [ ]:
# !rm -rf /content/.* /content/* || true

In [ ]:
!nvidia-smi

# --- Install and Setup ---
!pip install -q ultralytics

# Unzip dataset to two locations: 'dataset' and 'custom_data'
!unzip -q /content/data-4.zip -d /content/dataset
!unzip -q /content/data-4.zip -d /content/custom_data

# Check contents
!ls /content/dataset

# Download helper script for train-validation split
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# Run the split script (90% training, 10% validation)
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9

# --- Generate data.yaml dynamically ---
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
    """Creates a data.yaml config file from classes.txt"""
    if not os.path.exists(path_to_classes_txt):
        print(f'classes.txt file not found! Please create one at {path_to_classes_txt}')
        return

    with open(path_to_classes_txt, 'r') as f:
        classes = [line.strip() for line in f if line.strip()]
    number_of_classes = len(classes)

    data = {
        'path': '/content/data',
        'train': 'train/images',
        'val': 'validation/images',
        'nc': number_of_classes,
        'names': classes
    }

    with open(path_to_data_yaml, 'w') as f:
        yaml.dump(data, f, sort_keys=False)

    print(f'Created config file at {path_to_data_yaml}')

# Define paths and run the config generator
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'
create_data_yaml(path_to_classes_txt, path_to_data_yaml)

# Show contents of the generated YAML
print('\nFile contents:\n')
!cat /content/data.yaml

# --- Train YOLOv11s Model ---
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=60 imgsz=640

# --- Run Predictions on Validation Set and Save Outputs ---
!yolo detect predict model=runs/detect/train/weights/best.pt source=data/validation/images save=True

# --- Preview Some Prediction Outputs ---
import glob
from IPython.display import Image, display

for image_path in glob.glob('/content/runs/detect/predict/*.jpg')[:10]:
    display(Image(filename=image_path, height=400))
    print('\n')

# --- Load Trained Model and Print Class Names ---
from ultralytics import YOLO

model = YOLO('runs/detect/train/weights/best.pt')
print(model.names)

# --- Filter and Save Only Images with 'crown' Detections ---
from PIL import Image as PILImage, ImageDraw
import os

results = model.predict(source='data/validation/images', save=False)
os.makedirs('filtered_outputs', exist_ok=True)

for result in results:
    img_path = result.path
    img = PILImage.open(img_path).convert('RGB')
    draw = ImageDraw.Draw(img)

    boxes = result.boxes.xyxy.cpu()
    classes = result.boxes.cls.cpu().numpy()
    scores = result.boxes.conf.cpu().numpy()

    for box, cls_id, score in zip(boxes, classes, scores):
        if model.names[int(cls_id)] == 'crown':
            x1, y1, x2, y2 = map(int, box)
            label = f"crown {score:.2f}"
            draw.rectangle([x1, y1, x2, y2], outline='blue', width=3)
            draw.text((x1, y1 - 10), label, fill='blue')

    output_path = os.path.join('filtered_outputs', os.path.basename(img_path))
    img.save(output_path)

# --- Display Filtered Outputs ---
from IPython.display import Image, display

for image_path in glob.glob('filtered_outputs/*.png'):
    display(Image(filename=image_path, height=400))
    print('\n')

# # --- Optional: Run Prediction on a Specific Image ---
# results = model.predict(source='/content/test.jpeg', save=False)

# for result in results:
#     img = PILImage.open(result.path).convert("RGB")
#     draw = ImageDraw.Draw(img)

#     for box, cls, conf in zip(result.boxes.xyxy, result.boxes.cls, result.boxes.conf):
#         label = f"{model.names[int(cls)]} {conf:.2f}"
#         x1, y1, x2, y2 = map(int, box)
#         draw.rectangle([x1, y1, x2, y2], outline="black", width=2)
#         draw.text((x1, y1 - 10), label, fill="red")

#     display(img)

In [ ]:
import cv2
import pytesseract
import numpy as np
import pandas as pd

# Load the image
image = cv2.imread("1.png")

# Resize for better OCR (optional)
scale_percent = 200  # scale up for better OCR accuracy
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(image, dim, interpolation=cv2.INTER_LINEAR)

# Convert to grayscale for OCR
gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

# Use pytesseract to extract all displacement text
custom_config = r'--oem 3 --psm 6 outputbase digits'
text = pytesseract.image_to_string(gray, config=custom_config)

# Clean and parse numeric values
lines = text.splitlines()
displacements = []
for line in lines:
    try:
        val = float(line.strip())
        displacements.append(val)
    except:
        continue

# Identify color band area (hardcoded cropping based on image)
# Adjust these if your layout changes
color_bar_area = resized[50:-50, 20:60]  # Crop to focus only on color bar
num_bands = len(displacements)

# Calculate height per band
band_height = color_bar_area.shape[0] // num_bands

# Extract RGB for each band
color_values = []
for i in range(num_bands):
    y_start = i * band_height
    y_end = (i + 1) * band_height
    band = color_bar_area[y_start:y_end, :]

    # Compute average color (mean RGB)
    mean_color = cv2.mean(band)[:3]  # drop alpha
    mean_color = tuple(int(c) for c in mean_color)
    color_values.append(mean_color)

# Combine results
# Remove first and last entries (extra info like min/max text)
displacements = displacements[1:-1]
color_values = color_values[1:-1]

# Combine results
data = []
for disp, (r, g, b) in zip(displacements, color_values):
    data.append({"Displacement": disp, "R": r, "G": g, "B": b})


# Save to CSV
df = pd.DataFrame(data)
df.to_csv("legend_rgb_values.csv", index=False)

print("✅ CSV saved as 'legend_rgb_values.csv'")

In [ ]:
import cv2
import numpy as np
import pandas as pd

# Load image
img = cv2.imread("2.png")

# Convert to LAB color space (better separation of luminance & color)
lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

# Edge detection to remove thin lines
edges = cv2.Canny(img, threshold1=50, threshold2=150)

# Dilate to emphasize lines
kernel = np.ones((3, 3), np.uint8)
dilated = cv2.dilate(edges, kernel, iterations=1)

# Invert to get mask for "non-line" (background) areas
mask = cv2.bitwise_not(dilated)

# Use the mask to get only the background pixels
bg_pixels = img[mask == 255]

# Compute average background RGB
avg_color = np.mean(bg_pixels, axis=0)
avg_color = tuple(int(c) for c in avg_color)
print("Estimated background RGB:", avg_color)

# Load legend RGB table
df = pd.read_csv("legend_rgb_values.csv")

# Compute Euclidean distance to each RGB entry
def color_distance(c1, c2):
    return np.sqrt(sum((a - b) ** 2 for a, b in zip(c1, c2)))

df["Distance"] = df.apply(lambda row: color_distance(avg_color, (row["R"], row["G"], row["B"])), axis=1)

# Get closest match
closest_row = df.loc[df["Distance"].idxmin()]
closest_disp = closest_row["Displacement"]
print(f"Closest Displacement: {closest_disp:.3f} m")
